**Author**: Siddhant Sutar

Import Pandas, Numpy, and ordered logistic regression module by Fabian Pedregosa (with a few tweaks) obtained from https://github.com/fabianp/minirank/blob/master/minirank/logistic.py, since Scikit-learn doesn't support it yet.

In [1]:
import pandas as pd
import numpy as np
from logistic import ordinal_logistic_fit, ordinal_logistic_predict

Read training and test data.

In [2]:
train = pd.read_csv("train_data.csv")
test = pd.read_csv("test_data.csv")

Import NLTK library and TfIdfVectorizer to handle plot data. Vectorize the plot keywords and store in a sparse CSR matrix.

In [3]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stopset = set(stopwords.words('english'))
vectorizer = TfidfVectorizer(use_idf=False, norm=False, strip_accents='ascii', stop_words=stopset, binary=True)
train["FullPlot"] = train["FullPlot"].fillna('')
plot_keywords = vectorizer.fit_transform(train.FullPlot)
keywords_list = vectorizer.get_feature_names()

In [5]:
plot_keywords

<50184x60376 sparse matrix of type '<type 'numpy.float64'>'
	with 1325307 stored elements in Compressed Sparse Row format>

Read feature vectors into a csv.

In [6]:
ratings = pd.read_csv("ratings.csv")
genres = pd.read_csv("genres.csv")
directors = pd.read_csv("directors.csv", usecols=['Row', 'Column'])
actors = pd.read_csv("actors.csv", usecols=['Row', 'Column'])

Get the list of unique actors and directors present in the sparse COO feature matrix.

In [7]:
actors_list = pd.unique(actors["Column"].values.ravel()).tolist()
directors_list = pd.unique(directors["Column"].values.ravel()).tolist()

Function to generate feature column with 1s and 0s for an actor/director predictor into a readable format (Pandas) for the linear regression model, since they are stored as sparse COO matrices.

In [8]:
def get_feature_col(df, name, prefix=''):
    fc = pd.DataFrame(train["ID"], columns=['ID'], dtype=np.int64)
    fc[name] = 0
    temp = df.loc[df['Column'].isin([name])]
    fc.ix[fc.ID.isin(temp.Row.tolist()), name] = 1
    fc = fc.rename(columns = {name : prefix + name})
    return fc[prefix + name]

Ordered logit regression model

In [9]:
g = ['Action', 'Adventure', 'Sci-Fi']
d = ['Tim Miller']
c = ['Morena Baccarin', 'Ryan Reynolds', 'Gina Carano', 'Ed Skrein']
keywords = [] #["corruption", "government", "girl"]
feature_cols = [genres[g]]
for each in d:
    feature_cols.append(get_feature_col(directors, each, 'dir_'))
for each in c:
    feature_cols.append(get_feature_col(actors, each))
for each in keywords:
    if each in keywords_list:
        feature_cols.append(pd.DataFrame(plot_keywords[:, keywords_list.index(each)].toarray().flatten(), columns=[each]))
X = pd.concat([each for each in feature_cols], axis=1)
y = train.OrderedRating
w, theta = ordinal_logistic_fit(X, y)

E:\Softwares\Anaconda\lib\site-packages\scipy\optimize\_minimize.py:366: RuntimeWarning: Method TNC does not use Hessian-vector product information (hessp).
  'information (hessp).' % method, RuntimeWarning)
logistic.py:226: OptimizeWarning: Unknown solver options: maxfun
  jac=f_grad, hessp=f_hess, options=options, callback=callback)


Ordered ratings: 0 = [0-5), 1 = [5-6), 2 = [6-7), 3 = [7-8), 4 = [8-9), 5 = [9-10)

In [10]:
pred = ordinal_logistic_predict(w, theta, np.ones(len(X.columns)))
print(pred)

3


In [17]:
import json
predicted_dict = json.load(open("predicted_logistic.json", "r"))
y = train.OrderedRating
print len(predicted_dict)

20600


Predict the test data.

In [16]:
for index, row in test.iterrows():
    if row["Title"] not in predicted_dict:
        feature_cols = []
        my_genres = row["Genre"].split(", ")
        for each in my_genres:
            feature_cols.append(genres[each])
        if pd.notnull(row["Director"]):
            my_directors = row["Director"].split(", ")
            for each in my_directors:
                if each in directors_list:
                    feature_cols.append(get_feature_col(directors, each, 'dir_'))
        if pd.notnull(row["Cast"]):            
            my_actors = row["Cast"].split(", ")
            for each in my_actors:
                if each in actors_list:
                    feature_cols.append(get_feature_col(actors, each))
        X = pd.concat([each for each in feature_cols], axis=1)
        w, theta = ordinal_logistic_fit(X, y)
        predicted_dict[row["Title"]] = int(ordinal_logistic_predict(w, theta, np.ones(len(X.columns))))
    else:
        pass

In [14]:
with open('predicted_logistic.json', 'w') as outfile:
    json.dump(predicted_dict, outfile)